In [1]:
# import libraries
import os
import numpy as np
from tqdm import tqdm

from ifum_stitch import Stitch
from ifum_maskopt import Mask
from ifum_rectify import Rectify
from ifum_calibrate import Calibrate
from ifum_stack import Stack

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# directory containing unprocessed files
directory = "C:\\Users\\daniel\\OneDrive - The University of Chicago\\Documents\\cool lamps\\summer_24\\IFUM data\\ut20240212\\"

# all files included in a single stack, repeat where necessary
# only include string in file that includes all files from single exposure
data_filenames = ["1156","1157","1158","1162","1163","1164","1168","1169","1170"]
arc_filenames = ["1161","1161","1161","1167","1167","1167","1172","1172","1172"]
flat_filenames = ["1160","1160","1160","1166","1166","1166","1171","1171","1171"]

# mask???
# mode LR,STD,HR
mode = "STD"

# bad masks (on scale 1-276)
bad_blues = [23]
bad_reds = []

# stars to use in WCS (list RA,Dec)
# all stars should be present in at least some dithers
wcs_stars = [[74.8322, -58.6579],
             [74.8279, -58.6548],
             [74.8314, -58.6526],
             [74.8254, -58.6572],
             [74.8303, -58.6543]]

# plot or not
plot = False

In [3]:
# assert statements!!!
bad_masks = [np.array(bad_blues)-1,np.array(bad_reds)-1]
wcs_stars = np.array(wcs_stars)
if mode == "STD":
    total_masks = 552
    mask_groups = 12
else:
    print("invalid mode")

<h1><strong><span style="color:purple">STITCH</h1>

<span style="color:orange"><strong>creates 2 files, one for each detector, for both the data and arc files. also produces a cosmic ray mask for the data.

In [4]:
# out directory where all files are stored
if not os.path.exists("out"):
    os.makedirs("out")

# stitch and create file
for file in tqdm(data_filenames+arc_filenames+flat_filenames):
    file_to_stitch = Stitch(directory,file,None,None,None,None,None)
    file_to_stitch.load_files()
    file_to_stitch.save_file()
print("stitched files saved")

100%|██████████| 27/27 [00:32<00:00,  1.21s/it]

stitched files saved


In [5]:
# use flat and median filter to get rid of internal bias
for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_bias = Stitch(directory,None,None,"b",datafilename,arcfilename,flatfilename)
    file_for_bias.bias_sub()
    file_for_bias = Stitch(directory,None,None,"r",datafilename,arcfilename,flatfilename)
    file_for_bias.bias_sub()
print("internal bias solved")

100%|██████████| 9/9 [00:47<00:00,  5.30s/it]

internal bias solved


In [6]:
# create cosmic ray masks
for datafilename in tqdm(data_filenames):
    file_for_cmray = Stitch(directory,None,None,"b",datafilename,None,None)
    file_for_cmray.cmray_mask(data_filenames)
    file_for_cmray = Stitch(directory,None,None,"r",datafilename,None,None)
    file_for_cmray.cmray_mask(data_filenames)
print("cosmic ray masks created")

100%|██████████| 9/9 [15:00<00:00, 100.04s/it]

cosmic ray masks created


<h1><strong><span style="color:purple">OPTIMIZE MASK</h1>

<span style="color:orange"><strong>use current mask for single gaussian fits to get better mask guess  

In [ ]:
# first guess; complex guess
# IMPLEMENT RANSAC??? (HBDSCAN?)
for flatfilename in tqdm(np.unique(flat_filenames)):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    mask_polys0 = file_for_mask.first_guess(3)
    file_for_mask.mask_poly(mask_polys0,40)

    file_for_mask = Mask("r",flatfilename,bad_masks,total_masks,mask_groups)
    mask_polys0 = file_for_mask.first_guess(3)
    file_for_mask.mask_poly(mask_polys0,40)

  0%|          | 0/3 [00:00<?, ?it/s]c:\Users\daniel\OneDrive - The University of Chicago\Documents\cool lamps\IFUM_pipeline\ifum_maskopt.py:128: OptimizeWarning: Covariance of the parameters could not be estimated
  continuum,_ = scipy.optimize.curve_fit(self.f_2,cutoffs[0::6],flat_data[cutoffs[0::6],x])
100%|██████████| 3/3 [32:09<00:00, 643.04s/it]


In [4]:
center_deg = 5
sigma_deg = 3

for flatfilename in np.unique(flat_filenames):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.plot_trace_fits(center_deg,sigma_deg)

    file_for_mask = Mask("r",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.plot_trace_fits(center_deg,sigma_deg)    

In [9]:
sig_mult = 1.5

for flatfilename in tqdm(np.unique(flat_filenames)):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.get_flat_traces(center_deg,sigma_deg)
    file_for_mask.create_mask(sig_mult)
    
    file_for_mask = Mask("r",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.get_flat_traces(center_deg,sigma_deg)
    file_for_mask.create_mask(sig_mult)

100%|██████████| 3/3 [05:47<00:00, 115.79s/it]


In [4]:
# optimize arc files
expected_peaks = 15
sig_mult = 1.5

for arcfilename, flatfilename in tqdm(zip(np.unique(arc_filenames), np.unique(flat_filenames)),total=len(np.unique(arc_filenames))):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.optimize_trace(arcfilename,sig_mult,expected_peaks=expected_peaks)
    
    file_for_mask = Mask("r",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.optimize_trace(arcfilename,sig_mult,expected_peaks=expected_peaks)

100%|██████████| 3/3 [28:27<00:00, 569.08s/it]


In [5]:
# optimize data files
expected_peaks = 25
sig_mult = 1.5

for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.optimize_trace(datafilename,sig_mult,True,expected_peaks=expected_peaks)
    file_for_mask.get_rots(arcfilename,datafilename)
    
    file_for_mask = Mask("r",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.optimize_trace(datafilename,sig_mult,True,expected_peaks=expected_peaks)
    file_for_mask.get_rots(arcfilename,datafilename)

  0%|          | 0/9 [00:00<?, ?it/s]c:\Users\daniel\OneDrive - The University of Chicago\Documents\cool lamps\IFUM_pipeline\ifum_utils.py:125: RuntimeWarning: Mean of empty slice
  spectra = np.nanmean(cut_i,axis=0)
c:\Users\daniel\OneDrive - The University of Chicago\Documents\cool lamps\IFUM_pipeline\ifum_utils.py:125: RuntimeWarning: Mean of empty slice
  spectra = np.nanmean(cut_i,axis=0)
c:\Users\daniel\OneDrive - The University of Chicago\Documents\cool lamps\IFUM_pipeline\ifum_maskopt.py:357: OptimizeWarning: Covariance of the parameters could not be estimated
  popt,_ = scipy.optimize.curve_fit(ifum_utils.gauss,x[mask_area],intensities[i][mask_area],p0=p0)
 11%|█         | 1/9 [16:00<2:08:03, 960.38s/it]c:\Users\daniel\OneDrive - The University of Chicago\Documents\cool lamps\IFUM_pipeline\ifum_utils.py:125: RuntimeWarning: Mean of empty slice
  spectra = np.nanmean(cut_i,axis=0)
c:\Users\daniel\OneDrive - The University of Chicago\Documents\cool lamps\IFUM_pipeline\ifum_utils

In [6]:
# create masks for data
sig_mult = 1.5

for datafilename, flatfilename in tqdm(zip(data_filenames, flat_filenames),total=len(data_filenames)):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.create_mask(sig_mult,datafilename)
    
    file_for_mask = Mask("r",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.create_mask(sig_mult,datafilename)

100%|██████████| 9/9 [58:07<00:00, 387.47s/it]  


In [7]:
# create masks for arc!
sig_mult = 1.5

for arcfilename, flatfilename in tqdm(zip(np.unique(arc_filenames), np.unique(flat_filenames)),total=len(np.unique(arc_filenames))):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.create_mask(sig_mult,arcfilename)
    
    file_for_mask = Mask("r",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.create_mask(sig_mult,arcfilename)

100%|██████████| 3/3 [06:48<00:00, 136.13s/it]


<h1><strong><span style="color:purple">RECTIFY</h1>

<span style="color:orange"><strong>rectify + calibrate

In [ ]:
sig_mult = 1.5

for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_rect = Rectify("b",datafilename,arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_rect.optimize_centers("data",sig_mult)
    
    file_for_rect = Rectify("r",datafilename,arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_rect.optimize_centers("data",sig_mult)

for arcfilename, flatfilename in tqdm(zip(np.unique(arc_filenames), np.unique(flat_filenames)),total=len(np.unique(arc_filenames))):
    file_for_rect = Rectify("b","NA",arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_rect.optimize_centers("arc",sig_mult)    

    file_for_rect = Rectify("r","NA",arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_rect.optimize_centers("arc",sig_mult)    

In [4]:
for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_rect = Rectify("b",datafilename,arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_rect.rectify("data")

    file_for_rect = Rectify("r",datafilename,arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_rect.rectify("data")

for arcfilename, flatfilename in tqdm(zip(np.unique(arc_filenames), np.unique(flat_filenames)),total=len(np.unique(arc_filenames))):
    file_for_rect = Rectify("b","NA",arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_rect.rectify("arc")

    file_for_rect = Rectify("r","NA",arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_rect.rectify("arc")

  0%|          | 0/9 [00:00<?, ?it/s]

1 0.8292429292222714
1 0.8237829704496938
1 0.8085815255109997
1 0.8111318413025984
1 0.809169897587399
1 0.8005480740311685
1 0.8081694577051909
1 0.8001923530794502
1 0.7963900927466441
1 0.8051313432640839
1 0.8026505475080364
1 0.8062995961379041
1 0.8088330532327633
1 0.8036511482648635
1 0.8040251542789065
1 0.8006611191531033
1 0.8105157682913864
1 0.806049396539242
1 0.8357874870121488
1 0.8126718143715466
1 0.8140548599782429
1 0.8198876189136972
1 0.8179102587256151
1 0.8155110076308468
1 0.8221862146379325
0 0.6901203420143913
0 0.693478553997165
0 0.6807505013826591
0 0.6614258186288765
0 0.6562154660877466
0 0.651056767037609
0 0.6553687942214476
0 0.6532017778487371
BAD center
0 0.6563862796684667
0 0.6415414108856119
0 0.6372968187542499
0 0.6394600730040979
0 0.6371340823410624
0 0.6351890052938224
0 0.633795406152944
0 0.632641799730939
0 0.6358670014169389
0 0.6328026774862004
0 0.6324716531417856
0 0.634843564405109
0 0.636848189743269
0 0.6361322890566257
0 0.638693

 11%|█         | 1/9 [00:03<00:29,  3.75s/it]

1 0.8450318443808413
1 0.8304221865996322
1 0.8005767461612506
1 0.8100138522086493
1 0.8085236170995187
1 0.8044765788860617
1 0.8042294127021739
1 0.804482469499398
1 0.8043179569379961
1 0.8041857882939911
1 0.8120278924943806
1 0.8018512430235863
1 0.8074035649037518
1 0.8004604756150635
1 0.8029122875809686
1 0.8005546641122934
1 0.8084435022278659
1 0.801944663819445
1 0.8179120100018719
1 0.8103446744030645
1 0.8123504383139165
1 0.8099112864046181
1 0.815900912111683
1 0.8322623694860868
1 0.814932648677362
0 0.7127721709824693
0 0.6641728259959514
0 0.6823484280863424
0 0.684117499976304
0 0.6711154094766159
0 0.6534224931386164
0 0.6579503182147866
0 0.6578637226270858
0 0.6501769119670796
BAD center
0 0.6423450564575851
0 0.6371064780839439
0 0.6392764895850924
0 0.6350999970725191
0 0.6352330687689097
0 0.634774621575827
0 0.6336448255594134
0 0.6361832789863737
0 0.6316877965891189
0 0.6351862620586415
0 0.6269025771771282
0 0.6322129529214928
0 0.6298719990209963
0 0.6341

 22%|██▏       | 2/9 [00:07<00:25,  3.61s/it]

...

 33%|███▎      | 1/3 [00:02<00:04,  2.08s/it]

1 0.8124048064554549
1 0.8086383990190381
1 0.8027974007703566
1 0.8076195388755718
1 0.8002046351590139
1 0.803683817178659
1 0.7989612644175316
1 0.7969670244467804
1 0.7966217856062157
1 0.7940407098100551
1 0.7947736872709534
1 0.8025572590355669
1 0.8096652377055342
1 0.8056698908594966
1 0.8207899160001367
0 0.6751122825456213
0 0.6545345440649013
0 0.6609020791521403
0 0.6612522686356511
0 0.653560003227732
0 0.6526004754967906
0 0.6446181464193236
0 0.6435693020772127
0 0.64261250294335
0 0.6331779549578925
0 0.6345921198718919
0 0.6246128332725105
0 0.6175174302372193


 67%|██████▋   | 2/3 [00:03<00:01,  1.97s/it]

0 0.6213747978282099
0 0.6306346887914733
1 0.8123719901478994
1 0.8060440947086833
1 0.8084400385861659
1 0.8104896631172439
1 0.8042482761670487
1 0.8060160422910166
1 0.8021313547196945
1 0.7995793887370728
1 0.7963913105330589
1 0.7937823509294005
1 0.7981637476561751
1 0.8055645775079128
1 0.8103056207601612
1 0.8072209017648726
1 0.8140690037774818
0 0.6666755638175726
0 0.6515009841013079
0 0.653412199924727
0 0.6532254913824151
0 0.6446913645661713
0 0.6450256218793194
0 0.6375726663409018
0 0.6361755732528421
0 0.6354590751270387
0 0.6294100253470671
0 0.6268969613433918
0 0.6208863576204415
0 0.6129162466925573
0 0.6204357408885703
0 0.6297252781374105


100%|██████████| 3/3 [00:05<00:00,  1.98s/it]


In [5]:
# calibrate (rectified xs to wls)
for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_calib = Rectify("b",datafilename,arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_calib.calib()

    file_for_calib = Rectify("r",datafilename,arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_calib.calib()

100%|██████████| 9/9 [02:36<00:00, 17.34s/it]


In [6]:
# viz
for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_calib = Rectify("b",datafilename,arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_calib._viz()

    file_for_calib = Rectify("r",datafilename,arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_calib._viz()

100%|██████████| 9/9 [03:22<00:00, 22.53s/it]


<h1><strong><span style="color:purple">CALIBRATE</h1>

<span style="color:orange"><strong>calibrate

In [4]:
sig_mult = 1.5
bins = np.arange(7500,10000,1)

for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_calib = Calibrate(datafilename,arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_calib.get_spectra(sig_mult,bins,color="b")
    file_for_calib.get_spectra(sig_mult,bins,color="r")

  0%|          | 0/9 [00:00<?, ?it/s]

100%|██████████| 9/9 [3:17:53<00:00, 1319.24s/it]  


In [5]:
# intenisty calibrate all spectra, combine blue and red amplifiers

for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_calib = Calibrate(datafilename,arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_calib.intensity_corr()

100%|██████████| 9/9 [03:28<00:00, 23.19s/it]


In [5]:
# ONLY VIZ

for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_calib = Calibrate(datafilename,arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_calib._viz()

 11%|█         | 1/9 [00:27<03:41, 27.69s/it]


KeyboardInterrupt: 

<h1><strong><span style="color:purple">STACK</h1>

<span style="color:orange"><strong>stack dithers

In [6]:
# individually create datacubes to be used in dither stack

files_for_stack = Stack(data_filenames,bad_masks,total_masks,mask_groups,wcs_stars)
files_for_stack.hex_to_grid()
files_for_stack.spectra_to_datacube()

hexagon grid: 99.000,88.777
pixel grid: 99,89
8811 pixels


overlap percentages: 100%|██████████| 8811/8811 [00:21<00:00, 401.10it/s]


8691 (98.638%) of pixels overlap with hexagon grid
8183 (92.873%) of pixels overlap fully with hexagon grid


graphing pixels: 100%|██████████| 8811/8811 [00:09<00:00, 917.94it/s] 


plotting...


graphing pixels: 100%|██████████| 8811/8811 [00:10<00:00, 803.31it/s] 


plotting...


100%|██████████| 99/99 [01:32<00:00,  1.07it/s]


In [7]:
# transform data to WCS using cross-correlation
files_for_stack = Stack(data_filenames,bad_masks,total_masks,mask_groups,wcs_stars)
files_for_stack.wcs_datacubes()

calculting 2D cross-correlation shift guesses...
select 5 WCS stars in order; see popup window
optimizing WCS star coordinates...
performing WCS transforms...
	1156 used 5 reference stars
	1157 used 5 reference stars
	1158 used 5 reference stars
	1162 used 5 reference stars
	1163 used 4 reference stars
	1164 used 4 reference stars
	1168 used 5 reference stars
	1169 used 5 reference stars
	1170 used 5 reference stars
WCS transformations complete


In [8]:
# calibrate intensity for all datacubes
files_for_stack = Stack(data_filenames,bad_masks,total_masks,mask_groups,wcs_stars)
files_for_stack.full_intensity_callibration()

In [ ]:
# stack datacubes
files_for_stack = Stack(data_filenames,bad_masks,total_masks,mask_groups,wcs_stars)
files_for_stack.stack_datacubes()

100%|██████████| 99/99 [55:36<00:00, 33.70s/it]


final data cubes saved!
